From version 0.16 onwards Datashader supports rendering GeoPandas `GeoDataFrame`s directly rather than having to convert them to SpatialPandas first.

Here is a demonstration using the "geoda.natregimes" dataset from `geodatasets`, which includes data on US counties.

In [ ]:
import colorcet as cc
import datashader as ds
import datashader.transfer_functions as tf
import geopandas
from geodatasets import get_path

First load the GeoPandas `GeoDataFrame`. The first time this is called will download and cache the dataset, and subsequent calls will be faster as they will use the cached dataset.

In [ ]:
df = geopandas.read_file(get_path("geoda.natregimes"))
df.head()

The geometry type of this `GeoDataFrame` is POLYGON, and there are many columns. Columns that we will use are "DNL90" (log of population density in 1990) and "UE90" (unemployment rate in 1990).

## Population

To view 1990 population data using Datashader, first create a canvas to render to of an appropriate size.

In [ ]:
canvas = ds.Canvas(plot_width=800, plot_height=400)

The polygons are rasterized to the canvas using the `Canvas.polygons` method. This takes the source dataframe and name of the geometry column, plus an aggregator. Here we aggregate using the maximum of the population density column so that if there are multiple polygons touching a particular pixel it selects the maximum population density of those polygons.

In [ ]:
agg = canvas.polygons(df, geometry="geometry", agg=ds.max("DNL90"))

Now we shade the aggregation using Colorcet's `fire` colormap using histogram equalization so that the colors are applied nonlinearly for most even use of the colormap.

In [ ]:
im = tf.shade(agg, cmap=cc.fire, how="eq_hist")
tf.set_background(im, "black")

Counties with the lowest population density are rendered in black and dark red, and those with the highest population density are rendered in yellow and white.

## Unemployment

The "UE90" column contains unemployment percentage per county in 1990.

In [ ]:
df["UE90"], df["UE90"].min(), df["UE90"].max()

We can see from the `min` and `max` values that unemployment goes from zero to just over 30%.

To rasterize this using Datashader it is recommended to use a monochromatic colormap such as `colorcet.blues` and apply this linearly.

In [ ]:
agg = canvas.polygons(df, geometry="geometry", agg=ds.max("UE90"))
im = tf.shade(agg, cmap=cc.blues, how="linear")
tf.set_background(im, "white")

## Lines

Datashader can also render GeoPandas `GeoDataFrame`s as lines rather than polygons. Use the same code as in the population example above but replace `Canvas.polygons()` with `Canvas.line()` instead.

In [ ]:
agg = canvas.line(df, geometry="geometry", agg=ds.max("DNL90"))
im = tf.shade(agg, cmap=cc.fire, how="eq_hist")
tf.set_background(im, "black")

Lines can be rendered with antialiasing. Here is the previous example with an antialiased line width of 2 pixels.

In [ ]:
agg = canvas.line(df, geometry="geometry", agg=ds.max("DNL90"), line_width=2)
im = tf.shade(agg, cmap=cc.fire, how="eq_hist")
tf.set_background(im, "black")

## Geometry type support

The following table shows which geometry types are supported by which Datashader `Canvas` functions.

|Canvas function  |Supported geometry types                          |
|-----------------|--------------------------------------------------|
|`Canvas.line`    |LineString, MultiLineString, MultiPolygon, Polygon|
|`Canvas.point`   |MultiPoint, Point                                 |
|`Canvas.polygons`|MultiPolygon, Polygon                             |

## GeoPandas or SpatialPandas?

Datashader supports the same line, point and polygon rendering using GeoPandas and SpatialPandas, and produces the same output using either. They work in different ways such that SpatialPandas is usually faster for viewing large datasets and GeoPandas faster when zooming into a small region of a large dataset. The GeoPandas approach is more convenient if you already have your data in GeoPandas format and do not want the overhead of converting to SpatialPandas.